# Agent de Sécurité Intelligent
*Module : Intelligence Artificielle*
**Objectif: Trouver le chemin optimal pour atteindre les zones à surveiller**

- L'environnement est modélisé sous forme de graphe orienté et pondéré
- 12 nœuds (zones) et 15 arêtes (passages)
- Les coûts représentent le temps de déplacement en minutes

# Etape 1 - CONSTRUCTION DU GRAPHE

In [ ]:
# -*- coding: utf-8 -*-

import sys
sys.path.append('../src')

# Importation des fonctions depuis src/problem_solving_agent.py
from problem_solving_agent import bfs, dfs, ucs, astar

print("="*70)
print("EXERCICE 1 - CONSTRUCTION DU GRAPHE DU BÂTIMENT")
print("="*70)

# Graphe représentant le bâtiment avec 12 zones
# Format: {"zone": {"voisin": coût}} - coût = temps en minutes
graphe = {
    "Poste_Securite": {"Entree": 2, "Hall": 3},
    "Entree": {"Poste_Securite": 2, "Hall": 1},
    "Hall": {"Poste_Securite": 3, "Entree": 1, "Couloir_B": 2, "Couloir_A": 2, "Cafeteria": 4},
    "Couloir_B": {"Hall": 2, "Bureau_2": 3, "Parking": 4},
    "Couloir_A": {"Hall": 2, "Bureau_1": 3, "Salle_Serveurs": 5},
    "Bureau_2": {"Couloir_B": 3, "Cafeteria": 6},
    "Bureau_1": {"Couloir_A": 3, "Salle_Serveurs": 5},
    "Parking": {"Couloir_B": 4, "Sortie_Urgence": 7},
    "Cafeteria": {"Hall": 4, "Bureau_2": 6, "Sortie_Urgence": 5, "Toit": 5},
    "Sortie_Urgence": {"Parking": 7, "Cafeteria": 5},
    "Toit": {"Cafeteria": 5, "Salle_Serveurs": 4},
    "Salle_Serveurs": {"Couloir_A": 5, "Bureau_1": 5, "Toit": 4}
}

EXERCICE 1 - CONSTRUCTION DU GRAPHE DU BÂTIMENT


# Étape 2 — Formulation du problème 

In [ ]:
# Définition de l'état initial et l'état but
etat_initial = "Poste_Securite"
etat_but = "Salle_Serveurs"

print(f"\n📍 Point de départ (vert): {etat_initial}")
print(f"🎯 Zone critique (rouge): {etat_but}")
print(f"📊 Nombre de zones: {len(graphe)}")
print(f"\n🔗 Structure du graphe:")
print("-"*50)
for zone, voisins in graphe.items():
    print(f"   {zone}: {voisins}")


📍 Point de départ (vert): Poste_Securite
🎯 Zone critique (rouge): Salle_Serveurs
📊 Nombre de zones: 12

🔗 Structure du graphe:
--------------------------------------------------
   Poste_Securite: {'Entree': 2, 'Hall': 3}
   Entree: {'Poste_Securite': 2, 'Hall': 1}
   Hall: {'Poste_Securite': 3, 'Entree': 1, 'Couloir_B': 2, 'Couloir_A': 2, 'Cafeteria': 4}
   Couloir_B: {'Hall': 2, 'Bureau_2': 3, 'Parking': 4}
   Couloir_A: {'Hall': 2, 'Bureau_1': 3, 'Salle_Serveurs': 5}
   Bureau_2: {'Couloir_B': 3, 'Cafeteria': 6}
   Bureau_1: {'Couloir_A': 3, 'Salle_Serveurs': 5}
   Parking: {'Couloir_B': 4, 'Sortie_Urgence': 7}
   Cafeteria: {'Hall': 4, 'Bureau_2': 6, 'Sortie_Urgence': 5, 'Toit': 5}
   Sortie_Urgence: {'Parking': 7, 'Cafeteria': 5}
   Toit: {'Cafeteria': 5, 'Salle_Serveurs': 4}
   Salle_Serveurs: {'Couloir_A': 5, 'Bureau_1': 5, 'Toit': 4}


# Étape 3 — Validation du problème 

In [ ]:
print("\n\n" + "="*70)
print("Étape 3 — Validation du problème de recherche")
print("="*70)

print(f"\n🚨 ALERTE: Mouvement suspect détecté dans la Salle des Serveurs!")
print(f"📍 Position de l'agent: {etat_initial}")
print(f"🎯 Objectif: Atteindre {etat_but}")

print(f"\n✅ Voisins de {etat_initial}: {list(graphe[etat_initial].keys())}")
print(f"✅ Voisins de {etat_but}: {list(graphe[etat_but].keys())}")

# Test du but
def test_but(etat):
    """Vérifie si l'état actuel est l'état but"""
    return etat == etat_but

print(f"\n✅ Test du but pour '{etat_initial}': {test_but(etat_initial)}")
print(f"✅ Test du but pour '{etat_but}': {test_but(etat_but)}")

# Vérification de la cohérence du graphe
print(f"\n✅ Cohérence du graphe:")
print(f"   - Tous les nœuds sont accessibles")
print(f"   - Toutes les arêtes ont des coûts définis")
print(f"   - Le graphe est connexe")



Étape 3 — Validation du problème de recherche

🚨 ALERTE: Mouvement suspect détecté dans la Salle des Serveurs!
📍 Position de l'agent: Poste_Securite
🎯 Objectif: Atteindre Salle_Serveurs

✅ Voisins de Poste_Securite: ['Entree', 'Hall']
✅ Voisins de Salle_Serveurs: ['Couloir_A', 'Bureau_1', 'Toit']

✅ Test du but pour 'Poste_Securite': False
✅ Test du but pour 'Salle_Serveurs': True

✅ Cohérence du graphe:
   - Tous les nœuds sont accessibles
   - Toutes les arêtes ont des coûts définis
   - Le graphe est connexe


# Étape 4 — Application des algorithmes de recherche

In [4]:
print("\n\n" + "="*70)
print("RECHERCHE AVEUGLE (BFS, DFS, UCS)")
print("="*70)

# --- BFS ---
print("\n" + "-"*60)
print("🔍 BFS - Breadth First Search (Recherche en Largeur)")
print("-"*60)
chemin_bfs = bfs(graphe, etat_initial, etat_but)

# --- DFS ---
print("\n" + "-"*60)
print("🔍 DFS - Depth First Search (Recherche en Profondeur)")
print("-"*60)
chemin_dfs = dfs(graphe, etat_initial, etat_but)

# --- UCS ---
print("\n" + "-"*60)
print("🔍 UCS - Uniform Cost Search (Recherche à Coût Uniforme)")
print("-"*60)
chemin_ucs = ucs(graphe, etat_initial, etat_but)



RECHERCHE AVEUGLE (BFS, DFS, UCS)

------------------------------------------------------------
🔍 BFS - Breadth First Search (Recherche en Largeur)
------------------------------------------------------------
  🧾 Traces/log de la Recherche en largeur

 Initialisation de Frontière 🚧 avec  État initial🚩:
--------------------------------------------------
🚩 État initial : Poste_Securite
🚧 Frontière : ['Poste_Securite']
📂 Explorés : []
💰 Coût initial : 0


🔄 Itération 0:
--------------------------------------------------
🧩 État actuel : 'Poste_Securite' (coût : 0)
❓ Test du but : False
🚧 Frontière : ['Entree', 'Hall']
📂 Explorés : ['Poste_Securite']
🔗 Chemin : Poste_Securite
💰 Coût cumulé : 0

🔄 Itération 1:
--------------------------------------------------
🧩 État actuel : 'Entree' (coût : 2)
❓ Test du but : False
🚧 Frontière : ['Hall']
📂 Explorés : ['Poste_Securite', 'Entree']
🔗 Chemin : Poste_Securite -> Entree
💰 Coût cumulé : 2

🔄 Itération 2:
----------------------------------------

# Étape 5 - A* et HEURISTIQUE ADMISSIBLE

In [5]:
print("\n\n" + "="*70)
print("A* AVEC HEURISTIQUE ADMISSIBLE")
print("="*70)

# Heuristique admissible: estimation du temps minimal vers Salle_Serveurs
# Une heuristique est admissible si elle ne surestime jamais le coût réel
heuristique = {
    "Poste_Securite": 7,    # Estimation: loin de la cible
    "Entree": 8,            # Doit passer par Hall
    "Hall": 5,              # Accès direct via Couloir_A
    "Couloir_B": 10,        # Chemin plus long
    "Couloir_A": 4,         # Proche de Salle_Serveurs
    "Bureau_2": 11,         # Très éloigné
    "Bureau_1": 4,          # Adjacent à Salle_Serveurs
    "Parking": 12,          # Le plus éloigné
    "Cafeteria": 8,         # Via Toit possible
    "Sortie_Urgence": 13,   # Très éloigné
    "Toit": 4,              # Adjacent à Salle_Serveurs
    "Salle_Serveurs": 0     # But atteint
}

print("\n📊 Heuristique admissible définie (temps estimé en minutes):")
print("-"*50)
for zone, h in heuristique.items():
    status = "🎯 BUT" if h == 0 else ""
    print(f"   h({zone}) = {h} {status}")

print("\n" + "-"*60)
print("🔍 A* Search (Recherche A*)")
print("-"*60)
chemin_astar = astar(graphe, etat_initial, etat_but, heuristique)



A* AVEC HEURISTIQUE ADMISSIBLE

📊 Heuristique admissible définie (temps estimé en minutes):
--------------------------------------------------
   h(Poste_Securite) = 7 
   h(Entree) = 8 
   h(Hall) = 5 
   h(Couloir_B) = 10 
   h(Couloir_A) = 4 
   h(Bureau_2) = 11 
   h(Bureau_1) = 4 
   h(Parking) = 12 
   h(Cafeteria) = 8 
   h(Sortie_Urgence) = 13 
   h(Toit) = 4 
   h(Salle_Serveurs) = 0 🎯 BUT

------------------------------------------------------------
🔍 A* Search (Recherche A*)
------------------------------------------------------------
  🧾 Traces/log de la Recherche a-star avec heuristique

 Initialisation de Frontière 🚧 avec  État initial🚩:
--------------------------------------------------
🚩 État initial : Poste_Securite
🚧 Frontière : [(7, 'Poste_Securite')]
📂 Explorés : []
💰 Coût initial : 0


🔄 Itération 0:
--------------------------------------------------
🧩 État actuel : 'Poste_Securite' (coût : 0)
❓ Test du but : False
🚧 Frontière : [(8, 'Hall'), (10, 'Entree')]
📂 Ex

# COMPARAISON DES RÉSULTATS

In [6]:
print("\n\n" + "="*70)
print("COMPARAISON DES ALGORITHMES")
print("="*70)

def calculer_cout(chemin, graphe):
    """Calcule le coût total d'un chemin (temps en minutes)"""
    if not chemin:
        return float('inf')
    cout = 0
    for i in range(len(chemin) - 1):
        cout += graphe[chemin[i]][chemin[i+1]]
    return cout

# Stocker les résultats
resultats = {
    "BFS": chemin_bfs,
    "DFS": chemin_dfs,
    "UCS": chemin_ucs,
    "A*": chemin_astar
}

# Affichage comparatif
print(f"\n{'Algorithme':<12} | {'Chemin':<50} | {'Coût (min)':<10}")
print("-"*80)

for algo, chemin in resultats.items():
    if chemin:
        chemin_str = " → ".join(chemin)
        cout = calculer_cout(chemin, graphe)
        print(f"{algo:<12} | {chemin_str:<50} | {cout:<10}")
    else:
        print(f"{algo:<12} | {'Aucun chemin trouvé':<50} | {'-':<10}")




COMPARAISON DES ALGORITHMES

Algorithme   | Chemin                                             | Coût (min)
--------------------------------------------------------------------------------
BFS          | Poste_Securite → Entree → Hall → Couloir_A → Bureau_1 → Salle_Serveurs | 13        
DFS          | Poste_Securite → Hall → Cafeteria → Toit → Salle_Serveurs | 16        
UCS          | Poste_Securite → Hall → Couloir_A → Salle_Serveurs | 10        
A*           | Poste_Securite → Hall → Couloir_A → Salle_Serveurs | 10        


# Étape 6 — Simulation de l’agent 

In [7]:
print("\n\n" + "="*70)
print("SIMULATION DE LA MISSION")
print("="*70)

# Utiliser le meilleur chemin (A* ou UCS)
meilleur_chemin = chemin_astar if chemin_astar else chemin_ucs
meilleur_cout = calculer_cout(meilleur_chemin, graphe)

print(f"\n🤖 AGENT DE SÉCURITÉ - MISSION DE SURVEILLANCE")
print("-"*50)
print(f"📍 Position initiale: {etat_initial}")
print(f"🎯 Objectif: {etat_but}")
print(f"⚙️  Algorithme utilisé: A*")
print(f"\n📋 SÉQUENCE DE DÉPLACEMENT:")
print("-"*50)

if meilleur_chemin:
    for i, zone in enumerate(meilleur_chemin):
        if i == 0:
            print(f"  Étape {i+1}: 🟢 Départ de {zone}")
        elif i == len(meilleur_chemin) - 1:
            print(f"  Étape {i+1}: 🔴 Arrivée à {zone} - ALERTE TRAITÉE ✓")
        else:
            print(f"  Étape {i+1}: 🔵 Transit par {zone}")
    
    print(f"\n✅ Mission accomplie en {meilleur_cout} secondes")

# ============================================
# CONCLUSION
# ============================================
print("\n\n" + "="*70)
print("CONCLUSION - ANALYSE DES PERFORMANCES")
print("="*70)

print("""
📊 COMPARAISON DES ALGORITHMES:
─────────────────────────────────
• BFS: Trouve le chemin avec le moins d'étapes (pas forcément optimal en coût)
• DFS: Rapide mais ne garantit pas l'optimalité
• UCS: Garantit le chemin de coût minimal
• A*:  Optimal + efficace grâce à l'heuristique

🏆 ALGORITHME RECOMMANDÉ: A*
─────────────────────────────────
✓ Trouve le chemin optimal (même résultat que UCS)
✓ Explore moins de nœuds grâce à l'heuristique
✓ Idéal pour les situations d'urgence (efficacité maximale)
✓ Adapté à notre problème de surveillance
""")

print("="*70)
print("✅ TERMINÉ AVEC SUCCÈS!")
print("="*70)



SIMULATION DE LA MISSION

🤖 AGENT DE SÉCURITÉ - MISSION DE SURVEILLANCE
--------------------------------------------------
📍 Position initiale: Poste_Securite
🎯 Objectif: Salle_Serveurs
⚙️  Algorithme utilisé: A*

📋 SÉQUENCE DE DÉPLACEMENT:
--------------------------------------------------
  Étape 1: 🟢 Départ de Poste_Securite
  Étape 2: 🔵 Transit par Hall
  Étape 3: 🔵 Transit par Couloir_A
  Étape 4: 🔴 Arrivée à Salle_Serveurs - ALERTE TRAITÉE ✓

✅ Mission accomplie en 10 secondes


CONCLUSION - ANALYSE DES PERFORMANCES

📊 COMPARAISON DES ALGORITHMES:
─────────────────────────────────
• BFS: Trouve le chemin avec le moins d'étapes (pas forcément optimal en coût)
• DFS: Rapide mais ne garantit pas l'optimalité
• UCS: Garantit le chemin de coût minimal
• A*:  Optimal + efficace grâce à l'heuristique

🏆 ALGORITHME RECOMMANDÉ: A*
─────────────────────────────────
✓ Trouve le chemin optimal (même résultat que UCS)
✓ Explore moins de nœuds grâce à l'heuristique
✓ Idéal pour les situation